# Ebox Project
SWBio Data Science & Machine Learning Assessment.

## 1. Read in Fasta Files
Utilising code from Matt William's Biopython Course (1).

In [1]:
from Bio import SeqIO #import Seq10 module from Biopython package

Per1 = SeqIO.read("Per1.fasta", "fasta") # Read in fasta file using Seq10.read method
print(Per1) #look at Per1 gene SeqRecords

ID: NC_000077.7:68985934-69000791
Name: NC_000077.7:68985934-69000791
Description: NC_000077.7:68985934-69000791 Mus musculus strain C57BL/6J chromosome 11, GRCm39
Number of features: 0
Seq('GAAAATGACTGAGTTCTGAAATGCCAATGGTTGCTTTACAAGGGCTTGCCGGCT...CAA')


In [2]:
Per1_seq = Per1.seq #if you want just the sequence from this record
print(Per1_seq[0:20]) # print the first 20 nucleotides

GAAAATGACTGAGTTCTGAA


## 2. Search for E-Box element in Gene Sequence

### 2.1 Count the eboxes (or lack of eboxes!)
Using Biopython count method to search for subsequences within a sequence. It returns the number of times the subsequence is found.

In [3]:
Per1_seq.count("CACGTG") # Look for ebox sequence (CACGTG) in Per1 gene sequence

4

### 2.2 Create a ebox counter function:

In [4]:
# Create function requiring an argument (the fasta filename of the gene of interest)
def eboxcounter(filename): 
    gene = SeqIO.read(filename, "fasta") # read in file using seq10 module
    n = gene.seq.count("CACGTG") #count number of canonical eboxes present
    
    return n

In [5]:
x = eboxcounter("Per1.fasta") # put gene of interest filename here

print("The number of eboxes in this gene is:", x) # print out the number of eboxes in that gene

The number of eboxes in this gene is: 4


### 2.3 What is the position of these eboxes?

In [6]:
# Utilising nt_search() function 
import Bio.SeqUtils #import relevant package
ebox = "CACGTG"

x = Bio.SeqUtils.nt_search(str(Per1_seq), ebox) 
print(x) # Note this gives the position with python 0-indexing

['CACGTG', 2566, 3312, 3675, 5855]


In [7]:
print(Per1_seq[2566:2572]) # lets check how this did!

CACGTG


Alternatively, to get around python 0-indexing you can use Lana Caldarevic's find motif code (2).

In [8]:
# For loop running through the sequence one nucleotide at a time from start to finish
# looking for a position in the sequence that starts with CACGTG

s = Per1_seq # input gene of interest here
ebox = "CACGTG"

for position in range(len(s)): #range() function + len() function = number range the length of the string
    if s[position:].startswith(ebox): 
        print(position+1) # add 1 due to 0-based indexing 

2567
3313
3676
5856


### 2.4 Create a ebox position function:

In [9]:
# Create function requiring an argument (the fasta filename of the gene of interest)
def eboxposition(filename): 
    gene = SeqIO.read(filename, "fasta") # read in file using seq10 module
    s = gene.seq
    ebox = "CACGTG"
    x = Bio.SeqUtils.nt_search(str(s), ebox) 
    return x

In [10]:
eboxposition("Per1.fasta") # put gene of interest filename here

['CACGTG', 2566, 3312, 3675, 5855]

## 3. Research Questions
* Eboxes can be found in both intronic and promoter regions of a gene. You should find more eboxes if you include the promoter region of the Per1 gene, do you?
* Per1 is well-known example of a clock controlled gene. Try looking at some other genes?
* What if eboxes don't always look like that?

### 3.1 Compare number of eboxes when Per1 gene file includes promoter region

In [11]:
filelist = ["Per1.fasta", "Per1wProm.fasta"]

for file in filelist: #for loop to go through list of genes of interest
    x = eboxcounter(file) #use eboxcounter() function 
    print(x)

4
6


### 3.2 Looking at some other genes...

In [12]:
filelist = ["Per1.fasta", "Per1wProm.fasta", "Nnat.fasta", "NnatwProm.fasta", "Penk.fasta", "PenkwProm.fasta", "Vgf.fasta", "VgfwProm.fasta"] #list of genes of interest

values = [] #empty list to put ebox counts into

for file in filelist: #for loop to run through file list
    x = eboxcounter(file) #use eboxcounter function
    values.append(x) #add them to values list

print(values) #see values... but they're not easy to intepret on their own

[4, 6, 0, 0, 0, 0, 4, 4]


Make this into a dictionary...

In [17]:
# Combine filelist and values using the zip() and dict() functions to create a dictionary
# Code from Stack Overflow (3)
eboxdict = dict(zip(filelist, values)) 

print(eboxdict) #now values are next to their respective filenames

{'Per1.fasta': 4, 'Per1wProm.fasta': 6, 'Nnat.fasta': 0, 'NnatwProm.fasta': 0, 'Penk.fasta': 0, 'PenkwProm.fasta': 0, 'Vgf.fasta': 4, 'VgfwProm.fasta': 4}


In [18]:
# Use the dictionary to pull out ebox counts for specific gene e.g. Vgf
VgfEboxes = eboxdict["Vgf.fasta"]
print(VgfEboxes)

4


### 3.3 To look for imperfect ebox could use ntsearch() with N...

In [23]:
# Utilising nt_search() function 
import Bio.SeqUtils #import relevant package
eboxISH = "CACNTG" #use "N" to signify any nucleotide could be present in that position

x = Bio.SeqUtils.nt_search(str(Per1_seq), eboxISH) 
print(x) # Note this gives the position with python 0-indexing

['CAC[GATC]TG', 532, 2566, 2630, 3219, 3312, 3675, 4703, 5855, 6743, 10530, 11054, 11237, 11468]


In [24]:
Per1_seq[532: 538] #lets have a look

Seq('CACTTG')

## 4. References

1. Williams, M (n.d.) Biopython Course: SeqRecord objects [Source code]. https://milliams.com/courses/biopython/Input%20and%20Output.html
2. Caldarevic, L (2021) Find Motif [Source code]. https://github.com/0038lana/rosalind/blob/main/motif_dna/find_motif.ipynb
3. Hall, A (2015) Stack Overflow: How do I convert two lists into a dictionary? [Source code] https://stackoverflow.com/questions/209840/how-do-i-convert-two-lists-into-a-dictionary